In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Model
import numpy as np
import json

In [2]:
# Define hyperparameters
input_vocab_size = 114 # Vocabulary size for the source language
output_vocab_size = 80 # Vocabulary size for the target language
embedding_dim = 100  # Dimension of the GloVe embeddings
hidden_units = 32 # Number of units in LSTM layers
sequence_length = 35 # Maximum sequence length
batch_size = 32 # Batch size
num_epochs = 200 # Number of training epochs

In [3]:
glove_path = 'GloVe/glove.6B.100d.txt'  # Adjust the path to your downloaded GloVe file
embedding_matrix = {}  # Create an empty dictionary to store the embeddings

In [4]:
with open(glove_path, 'r', encoding='utf-8') as file:
    for line in file:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_matrix[word] = coefs #(coeff == Vectors)

In [5]:
# Create an embedding matrix for the source and target languages
source_embedding_matrix = np.zeros((input_vocab_size, embedding_dim))
target_embedding_matrix = np.zeros((output_vocab_size, embedding_dim))

In [6]:
##load the dataset
with open('card.json', 'r') as json_file:
    dataset = json.load(json_file)

In [7]:
#mapping input and output sequences to integers 
#input sequence = source_tokenizer
#output sequence = target_tokenizer

index_s = 1
index_t = 1
source_tokenizer = {}
target_tokenizer = {}


for i in dataset:
    question_toks = i["question_toks"]
    # print(query_toks)
    for j in question_toks:
        # print(j)
        
        if j not in source_tokenizer:
            source_tokenizer[j] = index_s
            index_s += 1
# print((frequency))
for i in dataset:
    query_toks = i["query_toks"]
    # print(query_toks)
    for j in query_toks:
        # print(j)
        
        if j not in target_tokenizer:
            target_tokenizer[j] = index_t
            index_t += 1

In [8]:
for word, i in source_tokenizer.items():
    embedding_vector = embedding_matrix.get(word)
    if embedding_vector is not None:
        source_embedding_matrix[i] = embedding_vector

for word, i in target_tokenizer.items():
    embedding_vector = embedding_matrix.get(word)
    if embedding_vector is not None:
        target_embedding_matrix[i] = embedding_vector

In [9]:
question_tokens = [example["question_toks"] for example in dataset]
query_tokens = [example["query_toks"] for example in dataset]

encoder_input_data_nopad = [[source_tokenizer[word] for word in sentence] for sentence in question_tokens]


decoder_input_data_nopad = [[target_tokenizer[word] for word in sentence] for sentence in query_tokens]


In [10]:
# Pad the sequences to a consistent length
def pad_sequences(sequences, max_length):
    padded_sequences = []
    for sequence in sequences:
        if len(sequence) < max_length:
            padded_sequence = sequence + [0] * (max_length - len(sequence))
        else:
            padded_sequence = sequence[:max_length]
        padded_sequences.append(padded_sequence)
    return padded_sequences

# Pad the encoder and decoder inputs
encoder_input_data = pad_sequences(encoder_input_data_nopad, sequence_length)
decoder_input_data = pad_sequences(decoder_input_data_nopad, sequence_length)
encoder_input_data = np.array(encoder_input_data)
decoder_input_data = np.array(decoder_input_data)

In [11]:
decoder_input_data

array([[ 1,  2,  3, ...,  0,  0,  0],
       [ 1,  2,  3, ...,  0,  0,  0],
       [ 1,  8,  9, ...,  0,  0,  0],
       ...,
       [ 1, 71,  6, ...,  0,  0,  0],
       [ 1,  8,  9, ...,  0,  0,  0],
       [ 1,  8,  9, ...,  0,  0,  0]])

In [12]:
#decoder_target_data
target_vocab = {
    0 : 0
}

decoder_target_data = np.zeros_like(decoder_input_data)
decoder_target_data[:, :-1] = decoder_input_data[:, 1:]
decoder_target_data[:, -1] = target_vocab[0]

In [13]:
# Define the encoder
encoder_inputs = tf.keras.layers.Input(shape=(sequence_length,))
encoder_embedding = Embedding(input_vocab_size, embedding_dim, weights=[source_embedding_matrix], trainable=False)(encoder_inputs)
encoder_lstm = LSTM(hidden_units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

In [14]:
# Define the decoder
decoder_inputs = tf.keras.layers.Input(shape=(sequence_length,))
decoder_embedding = Embedding(output_vocab_size, embedding_dim, weights=[target_embedding_matrix], trainable=False)(decoder_inputs)
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(output_vocab_size, activation='softmax')
output = decoder_dense(decoder_outputs)

In [15]:
# Build and compile the model
model = Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 35)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 35)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 35, 100)              11400     ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 35, 100)              8000      ['input_2[0][0]']             
                                                                                              

In [17]:
print(decoder_target_data.shape)
print(encoder_input_data.shape)
print(decoder_input_data.shape)

(80, 35)
(80, 35)
(80, 35)


In [18]:
# Train the model
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=num_epochs, validation_split=0.2)

Epoch 1/200
2/2 [==============================] - 6s 1s/step - loss: 4.3816 - val_loss: 4.3723
Epoch 2/200
2/2 [==============================] - 0s 68ms/step - loss: 4.3590 - val_loss: 4.3548
Epoch 3/200
2/2 [==============================] - 0s 59ms/step - loss: 4.3368 - val_loss: 4.3370
Epoch 4/200
2/2 [==============================] - 0s 67ms/step - loss: 4.3140 - val_loss: 4.3176
Epoch 5/200
2/2 [==============================] - 0s 66ms/step - loss: 4.2893 - val_loss: 4.2953
Epoch 6/200
2/2 [==============================] - 0s 67ms/step - loss: 4.2613 - val_loss: 4.2686
Epoch 7/200
2/2 [==============================] - 0s 60ms/step - loss: 4.2293 - val_loss: 4.2354
Epoch 8/200
2/2 [==============================] - 0s 64ms/step - loss: 4.1896 - val_loss: 4.1931
Epoch 9/200
2/2 [==============================] - 0s 64ms/step - loss: 4.1414 - val_loss: 4.1376
Epoch 10/200
2/2 [==============================] - 0s 65ms/step - loss: 4.0809 - val_loss: 4.0635
Epoch 11/200
2/2 [===

Epoch 84/200
2/2 [==============================] - 0s 56ms/step - loss: 1.5461 - val_loss: 1.3699
Epoch 85/200
2/2 [==============================] - 0s 63ms/step - loss: 1.5311 - val_loss: 1.3695
Epoch 86/200
2/2 [==============================] - 0s 66ms/step - loss: 1.5235 - val_loss: 1.3654
Epoch 87/200
2/2 [==============================] - 0s 59ms/step - loss: 1.5117 - val_loss: 1.3626
Epoch 88/200
2/2 [==============================] - 0s 57ms/step - loss: 1.5027 - val_loss: 1.3611
Epoch 89/200
2/2 [==============================] - 0s 58ms/step - loss: 1.4930 - val_loss: 1.3594
Epoch 90/200
2/2 [==============================] - 0s 49ms/step - loss: 1.4841 - val_loss: 1.3578
Epoch 91/200
2/2 [==============================] - 0s 59ms/step - loss: 1.4767 - val_loss: 1.3574
Epoch 92/200
2/2 [==============================] - 0s 57ms/step - loss: 1.4685 - val_loss: 1.3566
Epoch 93/200
2/2 [==============================] - 0s 54ms/step - loss: 1.4590 - val_loss: 1.3547
Epoch 94/2

2/2 [==============================] - 0s 61ms/step - loss: 1.0799 - val_loss: 1.3314
Epoch 167/200
2/2 [==============================] - 0s 53ms/step - loss: 1.0775 - val_loss: 1.3353
Epoch 168/200
2/2 [==============================] - 0s 60ms/step - loss: 1.0737 - val_loss: 1.3367
Epoch 169/200
2/2 [==============================] - 0s 61ms/step - loss: 1.0692 - val_loss: 1.3359
Epoch 170/200
2/2 [==============================] - 0s 46ms/step - loss: 1.0659 - val_loss: 1.3367
Epoch 171/200
2/2 [==============================] - 0s 66ms/step - loss: 1.0625 - val_loss: 1.3343
Epoch 172/200
2/2 [==============================] - 0s 56ms/step - loss: 1.0589 - val_loss: 1.3380
Epoch 173/200
2/2 [==============================] - 0s 56ms/step - loss: 1.0547 - val_loss: 1.3422
Epoch 174/200
2/2 [==============================] - 0s 58ms/step - loss: 1.0509 - val_loss: 1.3406
Epoch 175/200
2/2 [==============================] - 0s 54ms/step - loss: 1.0471 - val_loss: 1.3410
Epoch 176/200
